In [1]:
import time, os, re, math, numpy as np, pandas as pd, sklearn as sk, matplotlib as plt
# import sqlite3
from pandasql import sqldf
import seaborn as sns
pysqldf = lambda q: sqldf(q, globals())

np.random.seed(42)
from pandas import get_dummies
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold,\
cross_val_score, cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc,\
classification_report, make_scorer, recall_score, mean_squared_error,\
precision_recall_curve, precision_score, auc
from sklearn.metrics import roc_auc_score as ras
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,\
StandardScaler, MinMaxScaler, normalize
from imblearn.over_sampling import SMOTE, RandomOverSampler
import warnings
warnings.filterwarnings('ignore')

## HPSA

In [2]:
# Load dataset for all Primary Care HPSA destinations with one record per facility
df_hpsa_pc = pd.read_csv("data/hpsa/BCD_HPSA_FCT_DET_PC.csv")

In [3]:
df_hpsa_pc[["HPSA Component Name", "HPSA Geography Identification Number", "Common State County FIPS Code"]]

,HPSA Component Name,HPSA Geography Identification Number,Common State County FIPS Code
0,"Census Tract 48, Lucas County, Ohio",39095004800,39095
1,"Census Tract 46, Lucas County, Ohio",39095004600,39095
2,"Census Tract 47.01, Lucas County, Ohio",39095004701,39095
3,Noble Correctional Institution,POINT,39121
4,EDGERTON FAMILY HEALTH CENTER,POINT,39171
...,...,...,...
66853,Lipscomb,48295,48295
66854,SEMINOLE FAMILY MEDICAL CLINIC,POINT,48165
66855,Schleicher,48413,48413
66856,Baylor,48023,48023


In [4]:
df_hpsa_pc = df_hpsa_pc[(df_hpsa_pc['Designation Type']=="HPSA Population") | 
                        (df_hpsa_pc["Designation Type"]=="Geographic HPSA") |
                        (df_hpsa_pc["Designation Type"]=="High Needs Geographic HPSA")
                       ]

len(df_hpsa_pc)

59589

In [5]:
# filter to the targe years

df_hpsa_pc['HPSA Designation Year'] = df_hpsa_pc['HPSA Designation Date'].str[-4:].astype(int)
df_hpsa_pc['HPSA Designation Last Update Year'] = df_hpsa_pc['HPSA Designation Last Update Date'].str[-4:].astype(int)

df_hpsa_pc = df_hpsa_pc[df_hpsa_pc['HPSA Designation Last Update Year'] >= 2020]

df_hpsa_pc = df_hpsa_pc[df_hpsa_pc["HPSA Status"]=="Designated"]

In [6]:
df_hpsa_pc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16115 entries, 0 to 66857
Data columns (total 68 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   HPSA Name                                                                 16115 non-null  object 
 1   HPSA ID                                                                   16115 non-null  object 
 2   Designation Type                                                          16115 non-null  object 
 3   HPSA Discipline Class                                                     16115 non-null  object 
 4   HPSA Score                                                                16115 non-null  int64  
 5   PC MCTA Score                                                             16084 non-null  float64
 6   Primary State Abbreviation                                         

In [7]:
df_hpsa_pc.rename(columns={"Common State County FIPS Code":"CountyFIPS", "HPSA Geography Identification Number":"LocationID"}, inplace=True)

In [8]:
df_hpsa_county_scores = df_hpsa_pc.groupby("CountyFIPS")["HPSA Score"].mean().reset_index()
df_hpsa_county_scores.head()

,CountyFIPS,HPSA Score
0,01001,15.0
1,01003,14.0
2,01005,20.0
3,01007,10.0
4,01009,7.0


In [9]:
df_hpsa_census_scores = df_hpsa_pc.groupby("LocationID")["HPSA Score"].mean().reset_index()
df_hpsa_census_scores.head()

,LocationID,HPSA Score
0,01001,15.0
1,0100390207,14.0
2,0100390963,14.0
3,0100392754,14.0
4,0100393024,14.0


In [10]:
# df_hpsa_county_status = df_hpsa_pc.groupby("CountyFIPS")["HPSA Status"].value_counts()

# most_common_strings = df_hpsa_county_status.sort_values(ascending=False)

# # Get the index (most frequent string) for each group
# df_hpsa_county_status = most_common_strings.

# df_hpsa_county_status


In [11]:
df_hpsa_county_scores["CountyFIPS"] = pd.to_numeric(df_hpsa_county_scores["CountyFIPS"], errors='coerce')
df_hpsa_census_scores["LocationID"] = pd.to_numeric(df_hpsa_census_scores["LocationID"], errors='coerce')

## Places

In [12]:
df_places = pd.read_csv('data/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2023_release_20240418.csv')

In [13]:
df_places.head()

,Year,StateAbbr,StateDesc,CountyName,CountyFIPS,LocationName,DataSource,Category,Measure,Data_Value_Unit,...,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text
0,2021,AL,Alabama,Baldwin,1003,1003011300,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,29.8,40.9,4487,POINT (-87.87730464 30.43411562),1003011300,HLTHOUT,OBESITY,CrdPrv,Obesity
1,2021,AL,Alabama,Chambers,1017,1017954000,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,38.0,54.3,6669,POINT (-85.46010298 32.86128629),1017954000,HLTHOUT,OBESITY,CrdPrv,Obesity
2,2021,AL,Alabama,Cleburne,1029,1029959600,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,31.0,47.8,4202,POINT (-85.56228862 33.69258271),1029959600,HLTHOUT,OBESITY,CrdPrv,Obesity
3,2021,AL,Alabama,Covington,1039,1039962000,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,...,NaN,3.8,4.6,3917,POINT (-86.4651625 31.34674334),1039962000,HLTHOUT,STROKE,CrdPrv,Stroke
4,2021,AL,Alabama,Fayette,1057,1057020400,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,28.4,44.1,3550,POINT (-87.60702531 33.64745531),1057020400,HLTHOUT,OBESITY,CrdPrv,Obesity


In [14]:
df_places["Year"].value_counts()

Year
2021    1976988
2020     578125
Name: count, dtype: int64

In [15]:
df_places[["CountyName","CountyFIPS","LocationID"]].dtypes

CountyName    object
CountyFIPS     int64
LocationID     int64
dtype: object

In [16]:
df_healthout_county = df_places.groupby(["CountyFIPS", "Category", "MeasureId"])[["Data_Value"]].mean().reset_index()

# Pivot table with desired format
pivoted_df = df_healthout_county.pivot_table(values='Data_Value', index='CountyFIPS', columns='MeasureId')

# Convert pivot table to DataFrame (if desired)
pivoted_df_county = pivoted_df.reset_index()

pivoted_df_county

MeasureId,CountyFIPS,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,CHD,...,MAMMOUSE,MHLTH,MOBILITY,OBESITY,PHLTH,SELFCARE,SLEEP,STROKE,TEETHLOST,VISION
0,1001,11.100000,30.291667,14.958333,39.291667,79.225000,6.458333,10.483333,83.750000,6.141667,...,77.691667,18.491667,16.491667,40.241667,12.866667,4.325000,38.358333,3.291667,16.125000,5.341667
1,1003,9.458065,31.293548,16.025806,37.838710,80.809677,7.670968,9.929032,84.980645,6.922581,...,77.829032,16.564516,16.219355,38.158065,12.422581,3.754839,34.493548,3.329032,12.587097,4.622581
2,1005,17.244444,34.655556,12.111111,47.322222,81.455556,6.755556,11.755556,80.944444,8.733333,...,77.711111,20.477778,25.055556,43.566667,17.388889,7.722222,42.411111,5.100000,26.544444,9.722222
3,1007,13.875000,31.125000,15.125000,40.075000,78.750000,6.475000,10.575000,82.050000,6.875000,...,74.900000,19.575000,18.375000,39.400000,14.500000,4.950000,39.400000,3.625000,20.025000,6.175000
4,1009,13.122222,31.922222,15.011111,38.544444,79.044444,7.200000,10.400000,82.400000,7.266667,...,73.122222,18.755556,18.111111,38.411111,14.244444,4.522222,36.966667,3.500000,17.811111,5.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,14.283333,20.925000,18.366667,27.091667,70.916667,5.783333,9.616667,79.650000,4.650000,...,64.341667,16.475000,10.675000,34.225000,10.566667,3.050000,30.091667,2.391667,15.725000,4.125000
3138,56039,10.050000,19.075000,21.075000,23.350000,69.850000,5.875000,8.750000,84.575000,3.925000,...,74.500000,12.850000,7.675000,21.875000,7.925000,2.175000,26.450000,1.950000,8.725000,2.925000
3139,56041,13.933333,22.233333,16.433333,28.600000,71.466667,6.033333,9.866667,80.233333,5.200000,...,65.200000,16.333333,12.100000,34.433333,11.466667,3.566667,35.333333,2.700000,16.333333,4.500000
3140,56043,14.000000,27.733333,14.766667,32.800000,78.300000,7.933333,9.800000,80.333333,6.933333,...,66.833333,14.766667,14.666667,34.466667,12.233333,3.966667,30.166667,3.433333,16.366667,5.100000


In [17]:
df_healthout_census = df_places.groupby(["LocationID", "Category", "MeasureId"])[["Data_Value"]].mean().reset_index()

# Pivot table with desired format
pivoted_df = df_healthout_census.pivot_table(values='Data_Value', index='LocationID', columns='MeasureId')

# Convert pivot table to DataFrame (if desired)
pivoted_df_census = pivoted_df.reset_index()

pivoted_df_census

MeasureId,LocationID,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,CHD,...,MAMMOUSE,MHLTH,MOBILITY,OBESITY,PHLTH,SELFCARE,SLEEP,STROKE,TEETHLOST,VISION
0,1001020100,10.2,30.1,15.4,37.7,79.0,6.6,10.3,83.5,6.0,...,76.8,18.3,15.5,38.7,12.5,3.9,36.9,3.0,14.6,4.7
1,1001020200,13.7,28.8,13.9,42.2,78.8,5.3,11.3,84.6,5.4,...,81.0,19.4,17.8,45.0,13.2,5.0,43.4,3.6,19.4,6.5
2,1001020300,11.4,30.1,15.1,38.5,79.1,6.6,10.6,82.6,6.1,...,76.9,18.9,16.4,39.4,12.8,4.1,38.1,3.3,16.2,5.2
3,1001020400,7.9,32.0,15.0,38.3,81.7,8.4,9.3,85.9,6.5,...,75.5,15.2,14.2,34.8,10.9,3.0,33.4,3.1,10.9,3.6
4,1001020500,8.4,26.5,16.3,33.7,76.6,6.2,9.8,86.0,4.7,...,77.8,16.6,12.0,36.3,10.1,2.9,35.7,2.4,11.1,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72332,56043000200,11.8,28.2,14.8,33.1,78.7,8.2,9.6,81.7,6.8,...,67.7,14.0,13.8,34.2,11.8,3.7,29.8,3.3,14.4,4.5
72333,56043000301,15.8,25.1,16.0,29.9,75.8,7.0,10.0,79.5,6.1,...,65.9,15.7,13.6,34.1,11.9,3.7,30.7,3.1,17.1,5.0
72334,56043000302,14.4,29.9,13.5,35.4,80.4,8.6,9.8,79.8,7.9,...,66.9,14.6,16.6,35.1,13.0,4.5,30.0,3.9,17.6,5.8
72335,56045951100,12.9,26.4,16.5,33.0,76.5,7.5,9.7,79.9,6.7,...,60.7,15.2,14.2,36.0,12.6,4.1,33.9,3.3,17.7,5.0


In [18]:
pivoted_df_county["CountyFIPS"] = pd.to_numeric(pivoted_df_county["CountyFIPS"], errors='coerce')
pivoted_df_census["LocationID"] = pd.to_numeric(pivoted_df_census["LocationID"], errors='coerce')

In [19]:
df_places_counties = pivoted_df_county
df_places_census = pivoted_df_census

# ACS Census Age, Sex, Race, Income

In [20]:
df_acs = pd.read_csv('data/acs_combined.csv') # does not match at census tract level
# df_acs = pd.read_csv('data/ACSDP5Y2020_demo_housing-Data.csv', skiprows=[0]) # does not match at census tract level
# df_acs = pd.read_csv('data/ACS_2020_Household_Families-Data.csv', skiprows=[0]) # does not match at census tract level

In [21]:
df_acs.shape

(36568, 56)

In [22]:
df_acs.head(3)

,GeographyID_x,LocationID,CountyID,Geographic Area Name_x,Estimate_Households_Total,"Estimate_Households_Total_Less than $10,000","Estimate_Households_Total_$10,000 to $14,999","Estimate_Households_Total_$15,000 to $24,999","Estimate_Households_Total_$25,000 to $34,999","Estimate_Households_Total_$35,000 to $49,999",...,Percent_RACE_Total population_One race_Asian_Korean,Percent_RACE_Total population_One race_Asian_Vietnamese,Percent_RACE_Total population_One race_Asian_Other Asian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander,Percent_RACE_Total population_One race_Some other race,Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race)
0,0600000US0100190171,100190171,1001,"Autaugaville CCD, Autauga County, Alabama",1461,13.6,21.6,10.6,11.3,15.1,...,0,0,0,0,0,0,0,0,0,1.1
1,0600000US0100190315,100190315,1001,"Billingsley CCD, Autauga County, Alabama",1157,10.9,4.7,25.8,9.8,5,...,0,0,0,0,0,0,0,0,0.6,1.1
2,0600000US0100192106,100192106,1001,"Marbury CCD, Autauga County, Alabama",2320,6.7,1.5,11.2,9.3,21,...,0,0,0,0,0,0,0,0,0,1.8


In [23]:
df_acs_county = df_acs.drop(["GeographyID_x", "LocationID", "Geographic Area Name_x"], axis=1)

In [24]:
df_acs_census = df_acs

def convert_to_numeric(col):
  try:
    return pd.to_numeric(col, errors='coerce')  # Convert to NA on errors
  except:
    return col  # Keep original value if other errors occur

df_acs_county = df_acs_county.apply(convert_to_numeric)
df_acs_census = df_acs_census.apply(convert_to_numeric)

In [25]:
# df_acs_census.dropna(axis=0, inplace=True)

In [26]:
df_acs_census.shape

(36568, 56)

In [27]:
df_acs_county = df_acs_county.groupby("CountyID").mean().reset_index()
df_acs_county.head()

,CountyID,Estimate_Households_Total,"Estimate_Households_Total_Less than $10,000","Estimate_Households_Total_$10,000 to $14,999","Estimate_Households_Total_$15,000 to $24,999","Estimate_Households_Total_$25,000 to $34,999","Estimate_Households_Total_$35,000 to $49,999","Estimate_Households_Total_$50,000 to $74,999","Estimate_Households_Total_$75,000 to $99,999","Estimate_Households_Total_$100,000 to $149,999",...,Percent_RACE_Total population_One race_Asian_Korean,Percent_RACE_Total population_One race_Asian_Vietnamese,Percent_RACE_Total population_One race_Asian_Other Asian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander,Percent_RACE_Total population_One race_Some other race,Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race)
0,1001,5389.750000,9.075000,7.825000,14.825000,9.625000,13.225000,14.500000,10.050000,12.075000,...,0.125000,0.0750,0.00,0.025000,0.000000,0.025,0.0,0.0,0.350000,1.825000
1,1003,10505.875000,5.237500,4.925000,8.137500,12.037500,15.987500,16.300000,13.025000,14.162500,...,0.025000,0.0375,0.05,0.000000,0.000000,0.000,0.0,0.0,2.125000,5.637500
2,1005,1864.400000,11.520000,8.500000,19.380000,10.060000,13.380000,16.680000,7.460000,8.040000,...,0.020000,0.0600,0.06,0.020000,0.020000,0.000,0.0,0.0,2.480000,4.360000
3,1007,2419.666667,8.333333,4.733333,11.000000,7.666667,13.433333,19.266667,17.066667,12.333333,...,0.000000,0.0000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.033333,1.966667
4,1009,3029.285714,9.885714,4.271429,11.528571,12.071429,15.214286,17.114286,11.900000,11.257143,...,0.071429,0.0000,0.00,0.128571,0.028571,0.100,0.0,0.0,1.771429,10.871429


In [28]:
df_acs_census = df_acs_census.groupby("LocationID").mean().reset_index()
df_acs_census.head()

,LocationID,GeographyID_x,CountyID,Geographic Area Name_x,Estimate_Households_Total,"Estimate_Households_Total_Less than $10,000","Estimate_Households_Total_$10,000 to $14,999","Estimate_Households_Total_$15,000 to $24,999","Estimate_Households_Total_$25,000 to $34,999","Estimate_Households_Total_$35,000 to $49,999",...,Percent_RACE_Total population_One race_Asian_Korean,Percent_RACE_Total population_One race_Asian_Vietnamese,Percent_RACE_Total population_One race_Asian_Other Asian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander,Percent_RACE_Total population_One race_Some other race,Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race)
0,100190171,NaN,1001.0,NaN,1461.0,13.6,21.6,10.6,11.3,15.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1
1,100190315,NaN,1001.0,NaN,1157.0,10.9,4.7,25.8,9.8,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.1
2,100192106,NaN,1001.0,NaN,2320.0,6.7,1.5,11.2,9.3,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8
3,100192628,NaN,1001.0,NaN,16621.0,5.1,3.5,11.7,8.1,11.8,...,0.5,0.3,0.0,0.1,0.0,0.1,0.0,0.0,0.8,3.3
4,100390207,NaN,1003.0,NaN,9122.0,5.1,7.9,9.6,13.8,14.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,2.1


In [29]:
df_acs_county.rename(columns={"CountyID":"CountyFIPS"}, inplace=True)

In [30]:
df_acs_county["CountyFIPS"] = pd.to_numeric(df_acs_county["CountyFIPS"], errors='coerce')
df_acs_census["LocationID"] = pd.to_numeric(df_acs_census["LocationID"], errors='coerce')

In [31]:
# def clean_census(df):
#     df[['US_ID','LocationID']] = df['Geography'].str.split('S', expand=True)
#     df = df.apply(convert_to_numeric)
#     df.dropna(axis=1, inplace=True)
#     df=df.groupby("LocationID").mean().reset_index()
#     df["LocationID"] = pd.to_numeric(df["LocationID"], errors='coerce')
    
#     pattern = r"(?i)PERCENT"  # Pattern to match "Percent", "PERCENT", or "percent" (case-insensitive)

#     # Filter for columns that contain the pattern
#     filtered_cols = [col for col in df.columns if re.search(pattern, col) is not None]

#     # Select filtered columns
#     df = df[filtered_cols]
#     return df

# df_acs_census = clean_census(df_acs)

In [32]:
df_acs_census.shape

(36568, 56)

In [33]:
df_acs_census.head()

,LocationID,GeographyID_x,CountyID,Geographic Area Name_x,Estimate_Households_Total,"Estimate_Households_Total_Less than $10,000","Estimate_Households_Total_$10,000 to $14,999","Estimate_Households_Total_$15,000 to $24,999","Estimate_Households_Total_$25,000 to $34,999","Estimate_Households_Total_$35,000 to $49,999",...,Percent_RACE_Total population_One race_Asian_Korean,Percent_RACE_Total population_One race_Asian_Vietnamese,Percent_RACE_Total population_One race_Asian_Other Asian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander,Percent_RACE_Total population_One race_Some other race,Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race)
0,100190171,NaN,1001.0,NaN,1461.0,13.6,21.6,10.6,11.3,15.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1
1,100190315,NaN,1001.0,NaN,1157.0,10.9,4.7,25.8,9.8,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.1
2,100192106,NaN,1001.0,NaN,2320.0,6.7,1.5,11.2,9.3,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8
3,100192628,NaN,1001.0,NaN,16621.0,5.1,3.5,11.7,8.1,11.8,...,0.5,0.3,0.0,0.1,0.0,0.1,0.0,0.0,0.8,3.3
4,100390207,NaN,1003.0,NaN,9122.0,5.1,7.9,9.6,13.8,14.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,2.1


## CDC Deaths

In [34]:
df_cdc = pd.read_csv('data/cdc/Underlying Cause of Death, 2018-2021, Single Race (version 1).csv')

In [35]:
df_cdc.shape

(8244, 12)

In [36]:
df_cdc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8244 entries, 0 to 8243
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Notes                            0 non-null      float64
 1   County                           8244 non-null   object 
 2   County Code                      8244 non-null   int64  
 3   Drug/Alcohol Induced             8244 non-null   object 
 4   Drug/Alcohol Induced Code        8244 non-null   object 
 5   Drug/Alcohol Induced Cause       8244 non-null   object 
 6   Drug/Alcohol Induced Cause Code  8244 non-null   object 
 7   Deaths                           8244 non-null   int64  
 8   Population                       8244 non-null   int64  
 9   Crude Rate                       8244 non-null   object 
 10  pct_tot                          8244 non-null   float64
 11  Years                            8244 non-null   object 
dtypes: float64(2), int64

In [37]:
total = df_cdc.groupby(['County Code']).sum().reset_index()
total

,County Code,Notes,County,Drug/Alcohol Induced,Drug/Alcohol Induced Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,Population,Crude Rate,pct_tot,Years
0,1001,0.0,"Autauga County, ALAutauga County, ALAutauga Co...",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,2434,680130,8.8Unreliable1057.7,0.010736,2018-20212018-20212018-2021
1,1003,0.0,"Baldwin County, ALBaldwin County, ALBaldwin Co...",Drug-induced causesDrug-induced causesAlcohol-...,DDAO,Drug poisonings (overdose) Unintentional (X40-...,D1D2A9O9,10408,3639348,20.3Unreliable15.11106.7,0.011439,2018-20212018-20212018-20212018-2021
2,1005,0.0,"Barbour County, AL",All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,1385,99120,1397.3,0.013973,2018-2021
3,1007,0.0,"Bibb County, ALBibb County, AL",Drug-induced causesAll other non-drug and non-...,DO,Drug poisonings (overdose) Unintentional (X40-...,D1O9,1163,178814,Unreliable1280.7,0.013008,2018-20212018-2021
4,1009,0.0,"Blount County, ALBlount County, ALBlount Count...",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,3081,697758,16.810.31297.6,0.013247,2018-20212018-20212018-2021
...,...,...,...,...,...,...,...,...,...,...,...,...
3134,56037,0.0,"Sweetwater County, WYSweetwater County, WYSwee...",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,1463,509043,18.326.5817.4,0.008622,2018-20212018-20212018-2021
3135,56039,0.0,"Teton County, WY",All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,377,93617,402.7,0.004027,2018-2021
3136,56041,0.0,"Uinta County, WYUinta County, WYUinta County, WY",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,673,244125,UnreliableUnreliable787.7,0.008270,2018-20212018-20212018-2021
3137,56043,0.0,"Washakie County, WY",All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,383,31155,1229.3,0.012293,2018-2021


In [38]:
cdc_main = total[["County Code", "Deaths", "Population", "pct_tot"]]
cdc_main

,County Code,Deaths,Population,pct_tot
0,1001,2434,680130,0.010736
1,1003,10408,3639348,0.011439
2,1005,1385,99120,0.013973
3,1007,1163,178814,0.013008
4,1009,3081,697758,0.013247
...,...,...,...,...
3134,56037,1463,509043,0.008622
3135,56039,377,93617,0.004027
3136,56041,673,244125,0.008270
3137,56043,383,31155,0.012293


In [39]:
cdc_main['pct_tot'] = cdc_main['Deaths']/cdc_main['Population']
cdc_main

,County Code,Deaths,Population,pct_tot
0,1001,2434,680130,0.003579
1,1003,10408,3639348,0.002860
2,1005,1385,99120,0.013973
3,1007,1163,178814,0.006504
4,1009,3081,697758,0.004416
...,...,...,...,...
3134,56037,1463,509043,0.002874
3135,56039,377,93617,0.004027
3136,56041,673,244125,0.002757
3137,56043,383,31155,0.012293


In [40]:
col_names = ["CountyFIPS", "DrugOrAlc_Deaths", "Population", "DrugOrAlc_Deaths_pct"]
cdc_main.columns = col_names
cdc_main.head()

,CountyFIPS,DrugOrAlc_Deaths,Population,DrugOrAlc_Deaths_pct
0,1001,2434,680130,0.003579
1,1003,10408,3639348,0.002860
2,1005,1385,99120,0.013973
3,1007,1163,178814,0.006504
4,1009,3081,697758,0.004416


In [41]:
df_cdc_counties = cdc_main[["CountyFIPS", "DrugOrAlc_Deaths_pct"]]

In [42]:
df_cdc_counties["CountyFIPS"] = pd.to_numeric(df_cdc_counties["CountyFIPS"], errors='coerce')

## FTE Law Enforcement Officers

In [43]:
df_le = pd.read_excel('data/fbi_le/FTE_Law_Enforcement_Officers.xlsx', skiprows=[0,1,2])

In [44]:
df_le.head()

,State,County,Total law\nenforcement\nemployees,Total\nofficers,Total\ncivilians
0,ALABAMA-Metropolitan Counties,Autauga,43.0,32.0,11.0
1,NaN,Baldwin,319.0,135.0,184.0
2,NaN,Bibb,12.0,11.0,1.0
3,NaN,Blount,48.0,44.0,4.0
4,NaN,Colbert,57.0,31.0,26.0


In [45]:
df_hpsa_pc.columns

Index(['HPSA Name', 'HPSA ID', 'Designation Type', 'HPSA Discipline Class',
       'HPSA Score', 'PC MCTA Score', 'Primary State Abbreviation',
       'HPSA Status', 'HPSA Designation Date',
       'HPSA Designation Last Update Date', 'Metropolitan Indicator',
       'LocationID', 'HPSA Degree of Shortage', 'Withdrawn Date', 'HPSA FTE',
       'HPSA Designation Population', '% of Population Below 100% Poverty',
       'HPSA Formal Ratio', 'HPSA Population Type', 'Rural Status',
       'Longitude', 'Latitude', 'BHCMIS Organization Identification Number',
       'Break in Designation', 'Common County Name', 'Common Postal Code',
       'Common Region Name', 'Common State Abbreviation', 'CountyFIPS',
       'Common State FIPS Code', 'Common State Name', 'County Equivalent Name',
       'County or County Equivalent Federal Information Processing Standard Code',
       'Discipline Class Number', 'HPSA Address', 'HPSA City',
       'HPSA Component Name', 'HPSA Component Source Identification

In [46]:
df_le = df_le.groupby("County")["Total law\nenforcement\nemployees"].sum().reset_index()

In [47]:
df_le.head()

,County,Total law\nenforcement\nemployees
0,Abbeville,656.0
1,Acadia,104.0
2,Accomack,64.0
3,Ada,720.0
4,Adair,72.0


In [48]:
counties = df_hpsa_pc[["County Equivalent Name","Common County Name","CountyFIPS"]].drop_duplicates()

In [49]:
counties.rename(columns={"County Equivalent Name":"County"}, inplace=True)
counties.head()

,County,Common County Name,CountyFIPS
0,Lucas,"Lucas County, OH",39095
5,Cuyahoga,"Cuyahoga County, OH",39035
16,Van Wert,"Van Wert County, OH",39161
24,Mahoning,"Mahoning County, OH",39099
52,Huron,"Huron County, OH",39077


In [50]:
df_le_counties = df_le.merge(counties, on="County")

In [51]:
df_le_counties = df_le_counties[["CountyFIPS", "Total law\nenforcement\nemployees"]]

In [52]:
df_le_counties.rename(columns={"Total law\nenforcement\nemployees":"tot_law_enforce_FTE"}, inplace=True)

In [53]:
df_le_counties

,CountyFIPS,tot_law_enforce_FTE
0,45001,656.0
1,22001,104.0
2,51001,64.0
3,21001,72.0
4,40001,72.0
...,...,...
2304,48503,34.0
2305,06115,163.0
2306,04027,352.0
2307,08125,352.0


In [54]:
df_le_counties["CountyFIPS"] = pd.to_numeric(df_le_counties["CountyFIPS"], errors='coerce')

# FBI Crime by County

In [55]:
#get current directory where files are stored
# cwd = os.path.abspath('') 
cwd = os.chdir(r'C:\Users\elhartley\Documents\JAG_Hackathon_Team2\JAG-Hackathon-Team2\Data\Table 10 State Cuts')
files = os.listdir(cwd) 
#splice file list to only show the ones we're interested in
table_files = []

for file in files:
     if file.startswith('Table_10_Offenses'):
        table_files.append(file)

table_files[:5]

['Table_10_Offenses_Known_to_Law_Enforcement_Alabama_by_Metropolitan_and_Nonmetropolitan_Counties_2020.xlsx',
 'Table_10_Offenses_Known_to_Law_Enforcement_Arizona_by_Metropolitan_and_Nonmetropolitan_Counties_2020.xlsx',
 'Table_10_Offenses_Known_to_Law_Enforcement_Arkansas_by_Metropolitan_and_Nonmetropolitan_Counties_2020.xlsx',
 'Table_10_Offenses_Known_to_Law_Enforcement_California_by_Metropolitan_and_Nonmetropolitan_Counties_2020.xlsx',
 'Table_10_Offenses_Known_to_Law_Enforcement_Colorado_by_Metropolitan_and_Nonmetropolitan_Counties_2020.xlsx']

In [56]:
# Extract state names

state_names = []

for filename in table_files:
    state = filename.split('_')[7] 
    if state == "New" or state == "North" or state == "West" or state == "South":
        name2 = filename.split('_')[8]
        state = state + " " + name2
        state_names.append(state)
    else:
        state_names.append(state)

state_names[:5]

['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado']

In [57]:
state_names[0]

'Alabama'

In [58]:
state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',  # Not in the original list, but included for completeness
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',  # Not in the original list, but included for completeness
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',  # Not in the original list, but included for completeness
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',  # Not in the original list, but included for completeness
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# Convert state names to abbreviations using list comprehension
state_names = [state_abbrev[state] for state in state_names]

state_names[:5]

['AL', 'AZ', 'AR', 'CA', 'CO']

In [59]:
raw_df = pd.read_excel('Table_10_Offenses_Known_to_Law_Enforcement_Alabama_by_Metropolitan_and_Nonmetropolitan_Counties_2020.xlsx')
raw_df

,Table 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,ALABAMA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"by Metropolitan and Nonmetropolitan Counties, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[The data shown in this table do not reflect c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,Violent,NaN,NaN,NaN,NaN,Property,NaN,NaN,NaN
6,Metropolitan/Nonmetropolitan,County,Violent \nCrime,Murder and nonnegligent manslaughter,Rape2,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,Arson3
7,METROPOLITAN COUNTIES,Autauga,59,3,14,3,39,267,53,178,36,NaN
8,NONMETROPOLITAN COUNTIES,Cherokee,67,1,12,2,52,313,80,179,54,NaN
9,NaN,Clarke,42,2,6,2,32,53,27,17,9,NaN


In [60]:
def replace_missing_metropolitan(df):

  df['Metropolitan/Nonmetropolitan'].fillna(method='ffill', inplace=True)
  return df

In [61]:
df_test = pd.read_excel('Table_10_Offenses_Known_to_Law_Enforcement_Alabama_by_Metropolitan_and_Nonmetropolitan_Counties_2020.xlsx', skiprows=[0,1,2,3,4,5,6])
df_test = df_test.iloc[:, :-1]
df_test["State"] = state_names[0]
df_test = replace_missing_metropolitan(df_test.copy())
df_test.dropna(axis=0, inplace=True)
df_test

,Metropolitan/Nonmetropolitan,County,Violent \nCrime,Murder and nonnegligent manslaughter,Rape2,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,State
0,METROPOLITAN COUNTIES,Autauga,59.0,3.0,14.0,3.0,39.0,267.0,53.0,178.0,36.0,AL
1,NONMETROPOLITAN COUNTIES,Cherokee,67.0,1.0,12.0,2.0,52.0,313.0,80.0,179.0,54.0,AL
2,NONMETROPOLITAN COUNTIES,Clarke,42.0,2.0,6.0,2.0,32.0,53.0,27.0,17.0,9.0,AL
3,NONMETROPOLITAN COUNTIES,Marengo,26.0,6.0,2.0,1.0,17.0,88.0,27.0,42.0,19.0,AL


In [62]:
df_files = []
i = 0

#create a loop to add the appropriate year to the date column in each file
for file in table_files:
    df_file = pd.read_excel(file, skiprows=[0,1,2,3,4,5,6])
    df_file = df_file.iloc[:, :-1]
    
    # Define a regular expression to match digits
    pattern = r"\d+"

    # Replace digits in column names with an empty string
    df_file.columns = df_file.columns.str.replace(pattern, '', regex=True)
    
    df_file["State"] = state_names[i]
    df_file["Common County Name"] = df_file["County"] + " County, " + df_file["State"]
    df_file = replace_missing_metropolitan(df_file.copy())
    df_file.dropna(axis=0, inplace=True)
    df_files.append(df_file)
    i+=1
    
combined_df = pd.concat(df_files, ignore_index=True)

In [63]:
combined_df.head()

,Metropolitan/Nonmetropolitan,County,Violent \nCrime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,State,Common County Name
0,METROPOLITAN COUNTIES,Autauga,59.0,3.0,14.0,3.0,39.0,267.0,53.0,178.0,36.0,AL,"Autauga County, AL"
1,NONMETROPOLITAN COUNTIES,Cherokee,67.0,1.0,12.0,2.0,52.0,313.0,80.0,179.0,54.0,AL,"Cherokee County, AL"
2,NONMETROPOLITAN COUNTIES,Clarke,42.0,2.0,6.0,2.0,32.0,53.0,27.0,17.0,9.0,AL,"Clarke County, AL"
3,NONMETROPOLITAN COUNTIES,Marengo,26.0,6.0,2.0,1.0,17.0,88.0,27.0,42.0,19.0,AL,"Marengo County, AL"
4,METROPOLITAN COUNTIES,Mohave,153.0,11.0,10.0,7.0,125.0,1637.0,445.0,1018.0,174.0,AZ,"Mohave County, AZ"


In [64]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2134 entries, 0 to 2133
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Metropolitan/Nonmetropolitan          2134 non-null   object 
 1   County                                2134 non-null   object 
 2   Violent 
Crime                        2134 non-null   float64
 3   Murder and nonnegligent manslaughter  2134 non-null   float64
 4   Rape                                  2134 non-null   float64
 5   Robbery                               2134 non-null   float64
 6   Aggravated assault                    2134 non-null   float64
 7   Property 
Crime                       2134 non-null   float64
 8   Burglary                              2134 non-null   float64
 9   Larceny-theft                         2134 non-null   float64
 10  Motor vehicle theft                   2134 non-null   float64
 11  State            

In [65]:
counties["State"] = counties["Common County Name"].str[-2:]
counties[counties["County"]=="Cherokee"]

,County,Common County Name,CountyFIPS,State
996,Cherokee,"Cherokee County, SC",45021,SC
1119,Cherokee,"Cherokee County, NC",37039,NC
13778,Cherokee,"Cherokee County, KS",20021,KS
24315,Cherokee,"Cherokee County, IA",19035,IA
32378,Cherokee,"Cherokee County, OK",40021,OK
42440,Cherokee,"Cherokee County, AL",01019,AL
57649,Cherokee,"Cherokee County, TX",48073,TX


In [66]:
counties[counties["State"]=="MH"]

,County,Common County Name,CountyFIPS,State
45432,Rongelap,"Rongelap, MH",68350,MH
45433,Maloelap,"Maloelap, MH",68300,MH
45434,Likiep,"Likiep, MH",68180,MH
45435,Majuro,"Majuro, MH",68190,MH
45436,Bikar,"Bikar, MH",68060,MH
45437,Bikini,"Bikini, MH",68070,MH
45438,Ebon,"Ebon, MH",68080,MH
45439,Enewetak,"Enewetak, MH",68090,MH
45440,Mejit,"Mejit, MH",68310,MH
45441,Ailinginae,"Ailinginae, MH",68007,MH


In [67]:
combined_df[combined_df["County"]=="Majuro"]

,Metropolitan/Nonmetropolitan,County,Violent \nCrime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,State,Common County Name


In [68]:
counties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2679 entries, 0 to 66857
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   County              2679 non-null   object
 1   Common County Name  2679 non-null   object
 2   CountyFIPS          2679 non-null   object
 3   State               2679 non-null   object
dtypes: object(4)
memory usage: 104.6+ KB


In [69]:
# state_abbr = counties["State"].to_list()

match = []
no_match = []

for state in set(list(counties["State"].to_list())):
    if state in state_names:
        match.append(state)
    else:
        no_match.append(state)
        
print(set(no_match))
print(len(set(match)))
print(len(set(state_names)))
print(len(set(counties["State"].to_list())))


# AK, CT, RI, MA

{'CT', 'MH', 'AS', 'DC', 'MP', 'GU', 'MA', 'FM', 'VI', 'AK', 'PW', 'PR', 'RI'}
45
46
58


In [70]:
df_crime = combined_df.merge(counties, on="Common County Name")
df_crime = df_crime[['CountyFIPS', "Metropolitan/Nonmetropolitan",'Violent \nCrime',
       'Murder and nonnegligent manslaughter', 'Rape', 'Robbery',
       'Aggravated assault', 'Property \nCrime', 'Burglary', 'Larceny-theft',
       'Motor vehicle theft']]
df_crime.head()

,CountyFIPS,Metropolitan/Nonmetropolitan,Violent \nCrime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft
0,01001,METROPOLITAN COUNTIES,59.0,3.0,14.0,3.0,39.0,267.0,53.0,178.0,36.0
1,01019,NONMETROPOLITAN COUNTIES,67.0,1.0,12.0,2.0,52.0,313.0,80.0,179.0,54.0
2,01025,NONMETROPOLITAN COUNTIES,42.0,2.0,6.0,2.0,32.0,53.0,27.0,17.0,9.0
3,01091,NONMETROPOLITAN COUNTIES,26.0,6.0,2.0,1.0,17.0,88.0,27.0,42.0,19.0
4,04015,METROPOLITAN COUNTIES,153.0,11.0,10.0,7.0,125.0,1637.0,445.0,1018.0,174.0


In [71]:
df_crime = df_crime.drop_duplicates()
df_crime.shape

(1442, 11)

In [72]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1442 entries, 0 to 1441
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   CountyFIPS                            1442 non-null   object 
 1   Metropolitan/Nonmetropolitan          1442 non-null   object 
 2   Violent 
Crime                        1442 non-null   float64
 3   Murder and nonnegligent manslaughter  1442 non-null   float64
 4   Rape                                  1442 non-null   float64
 5   Robbery                               1442 non-null   float64
 6   Aggravated assault                    1442 non-null   float64
 7   Property 
Crime                       1442 non-null   float64
 8   Burglary                              1442 non-null   float64
 9   Larceny-theft                         1442 non-null   float64
 10  Motor vehicle theft                   1442 non-null   float64
dtypes: float64(9), ob

In [73]:
df_crime["CountyFIPS"] = pd.to_numeric(df_crime["CountyFIPS"], errors='coerce')

# Final Dataset

In [74]:
# at county level
combined_df = df_hpsa_county_scores.merge(df_places_counties, how="inner", on="CountyFIPS")
combined_df = combined_df.merge(df_le_counties, how="inner", on="CountyFIPS")
combined_df = combined_df.merge(df_acs_county, how="inner", on="CountyFIPS")
combined_df = combined_df.merge(df_crime, how='inner', on='CountyFIPS')
final_df_counties = combined_df.merge(df_cdc_counties, how="inner", on="CountyFIPS")

# at census level

# df_hpsa_places_census = df_hpsa_census_scores.merge(df_acs_census, how='inner', on ="LocationID")
# # df_hpsa_places_census = df_hpsa_census_scores.merge(df_places_census, how='inner', on ="LocationID")
# print(df_hpsa_places_census.shape)
# df_hpsa_acs_census = df_hpsa_places_census.merge(df_pop2_census, how='inner', on ="LocationID")
# print(df_hpsa_acs_census.shape)

In [75]:
# final_df_counties.head()

In [76]:
df = final_df_counties

def categorize_score(score):
  if score < 14:
    return 0
  else:
    return 1

df['HPSA Category'] = df['HPSA Score'].apply(categorize_score)
df.head()

,CountyFIPS,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,...,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,DrugOrAlc_Deaths_pct,HPSA Category
0,1001,15.000,11.100000,30.291667,14.958333,39.291667,79.225000,6.458333,10.483333,83.750000,...,3.0,14.0,3.0,39.0,267.0,53.0,178.0,36.0,0.003579,1
1,1019,11.000,12.116667,34.316667,14.100000,42.933333,82.283333,7.933333,10.600000,81.983333,...,1.0,12.0,2.0,52.0,313.0,80.0,179.0,54.0,0.007950,0
2,1025,13.000,14.400000,34.711111,11.922222,46.755556,82.788889,7.100000,11.455556,83.655556,...,2.0,6.0,2.0,32.0,53.0,27.0,17.0,9.0,0.007928,0
3,1091,19.000,12.983333,34.816667,11.750000,47.983333,82.866667,7.016667,11.433333,82.983333,...,6.0,2.0,1.0,17.0,88.0,27.0,42.0,19.0,0.016495,1
4,4007,17.625,13.306250,29.793750,15.075000,35.062500,80.087500,9.062500,10.918750,76.818750,...,6.0,4.0,2.0,174.0,370.0,52.0,236.0,82.0,0.004162,1


In [77]:
df.isnull().sum().to_list()

[0,
 0,
 59,
 59,
 59,
 59,
 59,
 59,
 59,
 0,
 59,
 59,
 59,
 59,
 0,
 59,
 0,
 0,
 59,
 0,
 59,
 59,
 59,
 59,
 59,
 59,
 59,
 59,
 59,
 0,
 59,
 59,
 59,
 59,
 59,
 0,
 59,
 0,
 59,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [78]:
print(df.shape)
df.dropna(axis=0, inplace=True)
df.shape

(1382, 104)


(1323, 104)

In [79]:
# df.to_csv('data/final_data_mean.csv')
df.to_csv('data/final_df_combined.csv')

# Building our model

In [80]:
df = pd.read_csv("data/final_df_combined.csv")

In [81]:
df

,Unnamed: 0,CountyFIPS,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,...,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,DrugOrAlc_Deaths_pct,HPSA Category
0,0,1001,15.000,11.100000,30.291667,14.958333,39.291667,79.225000,6.458333,10.483333,...,3.0,14.0,3.0,39.0,267.0,53.0,178.0,36.0,0.003579,1
1,1,1019,11.000,12.116667,34.316667,14.100000,42.933333,82.283333,7.933333,10.600000,...,1.0,12.0,2.0,52.0,313.0,80.0,179.0,54.0,0.007950,0
2,2,1025,13.000,14.400000,34.711111,11.922222,46.755556,82.788889,7.100000,11.455556,...,2.0,6.0,2.0,32.0,53.0,27.0,17.0,9.0,0.007928,0
3,3,1091,19.000,12.983333,34.816667,11.750000,47.983333,82.866667,7.016667,11.433333,...,6.0,2.0,1.0,17.0,88.0,27.0,42.0,19.0,0.016495,1
4,4,4007,17.625,13.306250,29.793750,15.075000,35.062500,80.087500,9.062500,10.918750,...,6.0,4.0,2.0,174.0,370.0,52.0,236.0,82.0,0.004162,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,1377,56029,9.000,10.020000,24.560000,15.300000,29.360000,77.420000,7.760000,9.380000,...,0.0,7.0,0.0,10.0,74.0,29.0,40.0,5.0,0.005807,0
1319,1378,56031,12.000,12.600000,29.200000,14.550000,36.400000,78.800000,8.600000,9.850000,...,0.0,1.0,0.0,1.0,48.0,2.0,39.0,7.0,0.007375,0
1320,1379,56035,14.000,12.100000,21.500000,19.050000,26.150000,71.650000,6.100000,9.250000,...,0.0,0.0,0.0,1.0,43.0,5.0,35.0,3.0,0.003848,1
1321,1380,56037,13.000,14.283333,20.925000,18.366667,27.091667,70.916667,5.783333,9.616667,...,0.0,7.0,1.0,8.0,76.0,10.0,54.0,12.0,0.002874,0


In [82]:
df.head()

,Unnamed: 0,CountyFIPS,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,...,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,DrugOrAlc_Deaths_pct,HPSA Category
0,0,1001,15.000,11.100000,30.291667,14.958333,39.291667,79.225000,6.458333,10.483333,...,3.0,14.0,3.0,39.0,267.0,53.0,178.0,36.0,0.003579,1
1,1,1019,11.000,12.116667,34.316667,14.100000,42.933333,82.283333,7.933333,10.600000,...,1.0,12.0,2.0,52.0,313.0,80.0,179.0,54.0,0.007950,0
2,2,1025,13.000,14.400000,34.711111,11.922222,46.755556,82.788889,7.100000,11.455556,...,2.0,6.0,2.0,32.0,53.0,27.0,17.0,9.0,0.007928,0
3,3,1091,19.000,12.983333,34.816667,11.750000,47.983333,82.866667,7.016667,11.433333,...,6.0,2.0,1.0,17.0,88.0,27.0,42.0,19.0,0.016495,1
4,4,4007,17.625,13.306250,29.793750,15.075000,35.062500,80.087500,9.062500,10.918750,...,6.0,4.0,2.0,174.0,370.0,52.0,236.0,82.0,0.004162,1


In [83]:
df.shape

(1323, 105)

In [87]:
df["Metropolitan/Nonmetropolitan"].replace({"METROPOLITAN COUNTIES":1, "NONMETROPOLITAN COUNTIES":0}, inplace=True)

In [88]:
df["Metropolitan/Nonmetropolitan"].value_counts()

Metropolitan/Nonmetropolitan
0    915
1    408
Name: count, dtype: int64

In [89]:
df.dropna(axis=0,inplace=True)
df.shape

(1323, 105)

In [90]:
df.head()

,Unnamed: 0,CountyFIPS,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,...,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property \nCrime,Burglary,Larceny-theft,Motor vehicle theft,DrugOrAlc_Deaths_pct,HPSA Category
0,0,1001,15.000,11.100000,30.291667,14.958333,39.291667,79.225000,6.458333,10.483333,...,3.0,14.0,3.0,39.0,267.0,53.0,178.0,36.0,0.003579,1
1,1,1019,11.000,12.116667,34.316667,14.100000,42.933333,82.283333,7.933333,10.600000,...,1.0,12.0,2.0,52.0,313.0,80.0,179.0,54.0,0.007950,0
2,2,1025,13.000,14.400000,34.711111,11.922222,46.755556,82.788889,7.100000,11.455556,...,2.0,6.0,2.0,32.0,53.0,27.0,17.0,9.0,0.007928,0
3,3,1091,19.000,12.983333,34.816667,11.750000,47.983333,82.866667,7.016667,11.433333,...,6.0,2.0,1.0,17.0,88.0,27.0,42.0,19.0,0.016495,1
4,4,4007,17.625,13.306250,29.793750,15.075000,35.062500,80.087500,9.062500,10.918750,...,6.0,4.0,2.0,174.0,370.0,52.0,236.0,82.0,0.004162,1


In [91]:
features = df.drop(['CountyFIPS', 'HPSA Score', 'HPSA Category'], axis=1)
# features = df.drop(['LocationID', 'HPSA Score', 'HPSA Category'], axis=1)
target = df['HPSA Category']

X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.3, stratify=target, random_state=42)

ros = RandomOverSampler(sampling_strategy='minority', random_state=42)
X_resampled, Y_resampled = ros.fit_resample(X_train, y_train)

recall_scores = []

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_val_scaled = scaler.transform(X_val)

models = [LogisticRegression(),
          DecisionTreeClassifier(),
          RandomForestClassifier(),
#           KNeighborsClassifier(),
          SVC(kernel='rbf', probability=True),
          XGBClassifier()]

for model in models:
    model.fit(X_train_scaled, Y_resampled)
    print(f'{model} : ')

    # Optional: Calculate and print training and validation ROC AUC scores
    # train_preds_proba = model.predict_proba(X_train_scaled)[:, 1]
    # print('Training ROC AUC Score : ', roc_auc_score(y_train, train_preds_proba))
    # val_preds_proba = model.predict_proba(X_val_scaled)[:, 1]
    # print('Validation ROC AUC Score : ', roc_auc_score(y_val, val_preds_proba))

    train_preds = model.predict(X_train_scaled)
    print('Training Accuracy : ', accuracy_score(Y_resampled, train_preds))

    val_preds = model.predict(X_val_scaled)
    print('Validation Accuracy : ', accuracy_score(y_val, val_preds))
    print()

    conf_matrix = confusion_matrix(y_val, val_preds, normalize='true')  # Normalized confusion matrix

    print("Confusion Matrix:")
    print(conf_matrix)

#     test_recall = recall_score(y_val, val_preds)
#     recall_scores.append(test_recall)
#     print(f"Recall: {test_recall}")

    print("")
    print("----------------------")
    print("")

LogisticRegression() : 
Training Accuracy :  0.7293666026871402
Validation Accuracy :  0.6322418136020151

Confusion Matrix:
[[0.65317919 0.34682081]
 [0.38392857 0.61607143]]

----------------------

DecisionTreeClassifier() : 
Training Accuracy :  1.0
Validation Accuracy :  0.6020151133501259

Confusion Matrix:
[[0.50289017 0.49710983]
 [0.32142857 0.67857143]]

----------------------

RandomForestClassifier() : 
Training Accuracy :  1.0
Validation Accuracy :  0.6599496221662469

Confusion Matrix:
[[0.6300578  0.3699422 ]
 [0.31696429 0.68303571]]

----------------------

SVC(probability=True) : 
Training Accuracy :  0.7850287907869482
Validation Accuracy :  0.654911838790932

Confusion Matrix:
[[0.67052023 0.32947977]
 [0.35714286 0.64285714]]

----------------------

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
         

In [92]:
features = df.drop(['CountyFIPS', 'HPSA Score', 'HPSA Category'], axis=1)
target = df['HPSA Category']

X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, stratify=target, random_state=42)

ros = RandomOverSampler(sampling_strategy='minority', random_state=42)
X_resampled, Y_resampled = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_val_scaled = scaler.transform(X_val)

lr_model = LogisticRegression()

lr_model.fit(X_train_scaled, Y_resampled)

# Retrieve the coefficients and feature names
coefficients = lr_model.coef_[0]
feature_names = X_train.columns

# Create a DataFrame to display the coefficients and feature names
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Odds Ratio': np.exp(coefficients)
})

# Sort the DataFrame by the absolute value of coefficients to highlight the most impactful features
feature_importance_df = feature_importance_df.reindex(feature_importance_df['Coefficient'].abs().sort_values(ascending=False).index)

# Display the feature importance DataFrame
print("Feature Importance:")
feature_importance_df.head(10)

Feature Importance:


,Feature,Coefficient,Odds Ratio
30,MOBILITY,1.784076,5.954076
6,CANCER,-1.419662,0.241796
12,COGNITION,-1.179704,0.307370
39,Estimate_Households_Total,1.059929,2.886167
20,DIABETES,-1.025222,0.358717
25,INDEPLIVE,1.021294,2.776786
14,COPD,0.952716,2.592741
22,GHLTH,-0.930275,0.394445
100,Motor vehicle theft,0.739337,2.094547
32,PHLTH,0.682484,1.978786


In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

feature_importances = clf.feature_importances_

In [ ]:
feature_names = X_train.columns  # Assuming feature names are in the DataFrame column names
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
feature_importance_df.sort_values(by='importance', ascending=False) # Sort by descending importance

# Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error  

In [ ]:
X = df.drop(['LocationID', 'HPSA Score', 'HPSA Category'], axis=1)
y = df['HPSA Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define a range of alpha values to explore
alpha_range = np.logspace(-3, 2, 6)  # Example: explore alpha from 0.001 to 100

param_grid = {'alpha': alpha_range}

In [ ]:
ridge_reg = Ridge()  # Create the base model

grid_search = GridSearchCV(estimator=ridge_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')  # Use negative MSE for minimization
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']

print(f"Best Alpha: {best_alpha}")

In [ ]:
alpha = 100.0  

ridge_reg = Ridge(alpha=alpha)
ridge_reg.fit(X_train, y_train)

In [ ]:
y_pred = ridge_reg.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")